# Implementation of Algorithm with SIFT

In [1]:
import gdal
import skimage
import numpy
from PIL import Image
ds = gdal.Open('1.tif')
width = ds.RasterXSize
height = ds.RasterYSize
gt = ds.GetGeoTransform()

from osgeo import osr
srs = osr.SpatialReference()
srs.ImportFromWkt(ds.GetProjection())

srsLatLong = srs.CloneGeogCS()
ct = osr.CoordinateTransformation(srs,srsLatLong)

In [2]:
from datetime import datetime

import numpy as np
from matplotlib import pyplot as plt
import cv2
from scipy import misc

def template_tracker(link_in='data.mp4', link_out='output_sift.mp4'):
    print("opening video file...")
    vs = cv2.VideoCapture(link_in)

    writer = None
    W = None
    H = None
    totalFrames = 0
    check=False
    coords=[]
    while True:
        frame = vs.read()
        frame = frame[1]
        
        if link_in is not None and frame is None:
            print("end")
            check=True
            break
        
        
        
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
        size = np.array(rgb.shape)
        #print(size)
        #new_size = tuple((size*0.75).astype(int))
        #new_size = (size[:2]*0.75)
        #new_size = cv2.normalize(new_size, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
        #rgb = skimage.transform.resize(rgb, new_size)
        #rgb = misc.imresize(rgb, 0.75)
        rgb = numpy.array(Image.fromarray(rgb).resize((int(0.75 * rgb.shape[1]), int(rgb.shape[0] * 0.75))
    ))
        #print(np.array(rgb.shape))
        
        if W is None or H is None:
            (H, W) = frame.shape[:2]

        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(link_out, fourcc, 30, (2448,1779), True)
        # Initiate SIFT detector
        sift = cv2.xfeatures2d.SIFT_create()
        MIN_MATCH_COUNT = 10
        
        if totalFrames % 256 == 0:
            
            img1 = rgb        # queryImage
            img2 = plt.imread('1.tif') # trainImage            
            
            # find the keypoints and descriptors with SIFT
            kp1, des1 = sift.detectAndCompute(img1,None)
            kp2, des2 = sift.detectAndCompute(img2,None)

            FLANN_INDEX_KDTREE = 0
            index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 2)
            search_params = dict(checks = 2)

            flann = cv2.FlannBasedMatcher(index_params, search_params)
            matches = flann.knnMatch(des1,des2,k=2)

            # store all the good matches as per Lowe's ratio test.
            good = []
            for m,n in matches:
                if m.distance < 0.7*n.distance:
                    good.append(m)
                    
            if len(good)>MIN_MATCH_COUNT:
                src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
                dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
                matchesMask = mask.ravel().tolist()

                h,w = img1.shape[:2]
                pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
                dst = cv2.perspectiveTransform(pts,M)
                points= np.int32(dst)
                #print(points)
                img2 = cv2.polylines(img2,[np.int32(dst)],True,255,5, cv2.LINE_8)
                
                M = cv2.moments(points)
                
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])
                
                img2= cv2.circle(img2, (cx, cy), 4, (0, 255, 255), 10)
                ax= gt[0] + cx*gt[1] + cy*gt[2]
                ay= gt[3] + cx*gt[4] + cy*gt[5]
                coord= ct.TransformPoint(ax,ay)
                lat= coord[0]
                long= coord[1]
                #print(long,lat)
                
            else:
                print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
                matchesMask = None
        
        draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

        img3 = cv2.drawMatches(img1,kp1,img2[:,:,:3],kp2,good,None,**draw_params)
        img3= cv2.putText(img3, str(lat)+' East', (10,1720), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        img3= cv2.putText(img3, str(long)+' North', (10,1650), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        coords.append([lat,long])
        
        nigg=cv2.cvtColor(img3, cv2.COLOR_RGB2BGR)
        #print(nigg.shape)
        if writer is not None:
            writer.write(nigg)
        cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
        cv2.imshow("Frame", nigg)
        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            break
            
        if check:
            break
            
        totalFrames += 1
    #print('done')
    writer.release()
    vs.release()
    cv2.destroyAllWindows()
    return coords
    #print(coordss)

In [3]:
start_time = datetime.now()
a=template_tracker(link_in='our_data1.mp4', link_out='outputkeremnaz.mp4')
end_time = datetime.now()
time1=end_time - start_time
print('Duration: {}'.format(time1))
print(a[:3])
print(len(a))

print(time1/len(a))


opening video file...
end
Duration: 0:00:05.592876
[[33.642891881458574, 72.99209837629381], [33.642891881458574, 72.99209837629381], [33.642891881458574, 72.99209837629381]]
56
0:00:00.099873


In [4]:
print(a[:5])

[[33.642891881458574, 72.99209837629381], [33.642891881458574, 72.99209837629381], [33.642891881458574, 72.99209837629381], [33.642891881458574, 72.99209837629381], [33.642891881458574, 72.99209837629381]]


In [5]:
print(b[:5])

NameError: name 'b' is not defined

In [ ]:
print(b[:5])